In [7]:
from tifffile import imread
import os
from matplotlib import pyplot as plt
import json
import geojson
from scipy.io import loadmat
import h5py
import numpy as np

In [8]:
# pth_WSI_segmentations = r'\\10.99.68.178\andreex\data\Stardist\PDAC model\maybe_all_tiles_donald\test_WSI\StarDist_2_9_24\json'
#pth_WSI_segmentations = r'\\10.99.68.178\andre\Eduarda PDAC\immune_cell_pipeline\star_dist_Donald_model\StarDist_1_29_23\json'

pth_WSI_segmentations = r'\\169.254.138.20\Andre_expansion\data\Stardist\monkey figs\ndpi images\jsons'
#npdi_im_path = r'\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity'

In [9]:
json_pth_list = sorted([os.path.join(pth_WSI_segmentations,file) for file in os.listdir(pth_WSI_segmentations) if file.endswith(".json")])
print(json_pth_list)

['\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0250.json', '\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0300.json', '\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0450.json', '\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0600.json', '\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0700.json', '\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0800.json', '\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0915.json']


In [10]:
def get_ds_data(segmentation_data, ds):
    data_list = []
    for data in segmentation_data:
        centroid = data['centroid'][0]
        contour = data['contour'][0]
    
        #print(centroid)
        #print(contour)
        
        ds_centroid = [int(c/ds) for c in centroid]
        ds_contour = [[value / ds for value in sublist] for sublist in contour]
        ds_contour = [[round(x,2), round(y,2)] for x, y in zip(ds_contour[0], ds_contour[1])]
        #ds_contour = ds_contour[0:-1:4]  # make shape have 8 points instead of 32
        
        #print(ds_centroid)
        #print(ds_contour)
        
        dat = [ds_centroid, ds_contour]
        data_list.append(dat)
    return data_list

In [11]:
json_pth_list[0]

'\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0250.json'

In [ ]:
# out_pth = r'\\10.99.68.178\andreex\data\Stardist\PDAC model\maybe_all_tiles_donald\test_WSI\StarDist_2_9_24\json\partial_jsons'

# out_pth = r'\\169.254.138.20\Andre_expansion\data\Stardist\monkey figs\ndpi images\32_polys_20x'
# ds_amt = 1 # 20x

out_pth = r'\\169.254.138.20\Andre_expansion\data\Stardist\monkey figs\ndpi images\32_polys_10x'
ds_amt = 1/0.4416 # 10x

# ds_amt = 4 # 2.5x
# ds_amt = 2/0.4416  # 5x
# ds_amt = 1/0.4416 # 10x
# ds_amt = 1 # 20x

# for p, file in enumerate(json_pth_list):
for p, file in enumerate(json_pth_list[1:7]):
    nm = file.split('\\')[-1]
    new_fn = os.path.join(out_pth, nm[:-5] + '.geojson')
    print(f'{p} / {len(json_pth_list)}')
    print(nm)
    
    if not os.path.exists(new_fn):
    
        segmentation_data = json.load(open(file))
        
        data_list = get_ds_data(segmentation_data, ds_amt)
    
        GEOdata = []
        
        for j, (centroid, contour) in enumerate(data_list):
            
            #if j == 100000:
            #    break
            
            centroid = [centroid[0] + 0, centroid[1] + 0]
            # xy coordinates are swapped, so I reverse them here with xy[::-1]
            # note: add 1 to coords to fix 0 indexing vs 1 index offset
            contour = [[coord+0 for coord in xy[::-1]] for xy in contour]  # Convert coordinates to integers
            contour.append(contour[0]) # stardist doesn't close the circle, needed for qupath
        
            # Create a new dictionary for each contour
            dict_data = {
                "type": "Feature",
                "id": "PathCellObject",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [contour]
                },
                "properties": {
                    'objectType': 'annotation',
                    'classification': {'name': 'Nuclei', 'color': [97, 214, 59]}
                }
            }
        
            GEOdata.append(dict_data)
        
        with open(new_fn,'w') as outfile:
            geojson.dump(GEOdata,outfile)
        print('Finished',new_fn)
    
    else:
        print(f'skipping {new_fn}')

0 / 7
monkey_fetus_40_0300.json


In [19]:
json_pth_list[1:7]

['\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0300.json',
 '\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0450.json',
 '\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0600.json',
 '\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0700.json',
 '\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0800.json',
 '\\\\169.254.138.20\\Andre_expansion\\data\\Stardist\\monkey figs\\ndpi images\\jsons\\monkey_fetus_40_0915.json']